In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotly.express as px
from tqdm import tqdm
import random

import optuna
from optuna.samplers import RandomSampler
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.ERROR)

In [2]:
import sys
sys.path.append('../src')

import vrpp
import importlib
importlib.reload(vrpp)

<module 'vrpp' from '/home/chervovn04/Programming/hackathons/2023/optimal_encashement/notebooks/../src/vrpp.py'>

In [3]:
dist = pd.read_csv('../data/raw/times v4.csv')
le = preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])
dist.head()

,Origin_tid,Destination_tid,Total_Time,from_int,to_int
0,636538,683103,15.32,864,1354
1,636538,634763,16.20,864,624
2,636538,683128,16.27,864,1358
3,636538,683789,16.77,864,1374
4,636538,634709,17.67,864,603


In [4]:
data = pd.read_excel('../data/raw/terminal_data_hackathon v4.xlsx', 'Incomes')
data.head()

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
0,406136,160000,90000,105000,99000,107000,110000,60000,75000,89000,...,91000,78000,0,165000,0,189000,106000,94000,75000,74000
1,406139,387000,103000,206000,168000,124000,78000,165000,164000,174000,...,164000,153000,151000,157000,206000,182000,123000,138000,112000,179000
2,406145,287000,143000,136000,124000,117000,123000,140000,139000,138000,...,119000,100000,179000,169000,118000,118000,114000,128000,121000,124000
3,406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,...,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
4,406180,597000,96000,82000,71000,72000,86000,55000,55000,75000,...,82000,56000,70000,59000,105000,70000,77000,87000,59000,55000


In [5]:
config = {'num_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': 100,
          'terminal_service_persent': 0, #0.01
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10,
          'left_days_coef': 0,
          'encashment_coef': 0,}

In [6]:
cash = (data[data.columns[1:]].values * (1 + 0.2 * (random.random() - 0.5))).copy()
real_cash = data[data.columns[1:]].values.copy()
cash = real_cash.copy()

In [7]:
cash = cash.astype(int)
cash.shape

(1630, 92)

In [8]:
INF = 1e9

In [9]:
def get_cost(days_left):
    if days_left == 0: return INF
    return 2 ** (config['max_not_service_days'] - days_left)

In [ ]:
num_vehicles = 5
num_terminals = cash.shape[0]
time_until_force = [config['max_not_service_days'] - 1 for i in range(num_terminals)]
cur_cash = np.zeros(num_terminals)
hist = {'visited': [], 'loss': []}
bad = False

myvrp = vrpp.VRPP(dist, 10, 10 * 60, num_vehicles, solution_limit=100, time_limit=100, dead_loss=False)

days = 31
for day in tqdm(range(days)):
    mask = []
    cost = []
    for i in range(num_terminals):
        current_money = cur_cash[i]
        force = time_until_force[i]
        for forecast in range(force):
            if current_money > config['max_terminal_money']:
                force = forecast
                break
            if not forecast:
                current_money += cash[i][day + forecast]
            else:
                current_money += cash[i][day + forecast] # check later
        mask.append(1)
        cost.append(int(get_cost(force)))
    
    for i in range(100):
        print(" " * (4 - len(str(cost[i]))) + str(cost[i]), end=' ')
    print()
    # print(" "cost[:100])
    
    visited, paths = myvrp.find_vrp(cost, mask)
    hist['visited'].append(visited)
    
    prev = []
    cur_loss = 0
    for i in range(num_terminals):
        if cur_cash[i] > config['max_terminal_money'] or time_until_force[i] == 0:
            if not visited[i]:
                print('Dead')
                bad = True
                break

        if visited[i]:
            cur_loss += config['terminal_service_cost']
            cur_cash[i] = 0
            time_until_force[i] = 14


        time_until_force[i] -= 1 
        cur_loss += cur_cash[i] * config['persent_day_income']
        cur_cash[i] += real_cash[i][day]
    
    if bad: break
    
    hist['loss'].append(cur_loss)
print(sum(hist['loss']))

  0%|                                                                             | 0/31 [00:00<?, ?it/s]

   8  256  128    8  256  512  128 4096 4096  128  512   16 1024 4096 2048 4096 4096  512    2    2    2    2    2  128  256    2   16 1024    4  512    2  128    2    8    2  128    2    2  256    2    2   32    8    2   32   16    2    2    2    2    4    2 1024    2    2  256   32    2   32    2    2  256    2    2   16   16    2  128  256    2    2  256    2 1024    2    2    2    2    2    4   16    2    4    2    2    2    2  512   64    2    2    2    8    2    2   32    2    4    2    2 


  3%|██▏                                                                  | 1/31 [00:11<05:41, 11.37s/it]

  16  512  256   16  512 1024  256 8192 8192  256 1024   32 2048 8192 4096 8192 8192 1024    4    4    4    4    4  256  512    4   32 2048    8 1024    4  256    4   16    4  256    4    4  512    4    4   64   16    4   64   32    4    4    4    4    8    4 2048    4    4  512   64    4   64    4    4  512    4    4   32   32    4  256  512    4    4  512    4 2048    4    4    4    4    4    8   32    4    8    4    4    4    4 1024  128    4    4    4   16    4    4   64    4    8    4    4 


  6%|████▍                                                                | 2/31 [00:23<05:45, 11.91s/it]

...

 512  512  512  512   16 1024  256 2048  256   16  512   64  256   64    8 1024 8192  512  128  128  128  128  128   16  512  128   32 2048  256  128  128   16  128  512  128  256  128  128  512  128  128    4  512  128    4    2  128  128  128  128  256  128  512  128  128   16    8  128    8  128  128  256  128  128   16    4  128   32  128  128  128   16  128   64  128  128  128  128  128  256    8  128  256  128  128  128  128  256   32  128  128  128  512  128  128    2  128  256  128  128 


 23%|███████████████▌                                                     | 7/31 [04:40<18:16, 45.67s/it]

   4  256  128    2   32 1024  512 2048  512   32    8  128  512  128   16 1024 8192   32  256  256  256  256  256   32   64  256   64 2048  512  256  256   32  256    2  256  512  256  256   64  256  256    8    2  256    8    4  256  256  256  256  512  256  512  256  256   32   16  256   16  256  256   16  256  256   32    8  256   64  256  256  256   32  256  128  256  256  256  256  256  512   16  256  512  256  256  256  256  512   64  256  256  256    2  256  256    4  256  512  256  256 


 26%|█████████████████▊                                                   | 8/31 [05:08<15:17, 39.91s/it]

   8  512  256    4   64 1024  128 2048  128   64   16  256 1024  256   32 2048 8192   64  512  512  512  512  512   64  128  512   16 2048    8  512  512   64  512    4  512   32  512  512  128  512  512   16    4  512   16    8  512  512  512  512    2  512  512  512  512   64   32  512   32  512  512   32  512  512   64   16  512  128  512  512  512   64  512  256  512  512  512  512  512    2   32  512    2  512  512  512  512 1024  128  512  512  512    4  512  512    8  512 1024  512  512 
